## Multi-Task Classification Model for the PubChem datasets

To execute this notebook, first install the DeepChem library:

    conda create --name deepchem-test
    conda activate deepchem-test
    conda install -y -c conda-forge rdkit nb_conda_kernels matplotlib
    pip3 install tensorflow==2.2.0
    pip3 install --pre deepchem

In [1]:
from rdkit import Chem, DataStructs
from rdkit.Chem import rdFingerprintGenerator
from rdkit.Chem import AllChem
from rdkit import SimDivFilters
from collections import defaultdict
import pandas as pd
import numpy as np
import gzip
import pickle
import matplotlib.pyplot as plt
import matplotlib as mpl
import deepchem as dc
from rdkit.Chem.MolStandardize import rdMolStandardize



In [2]:
import sklearn
import random
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.utils import resample



In [3]:
from rdkit import RDLogger
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)

## Functions

In [4]:
# Remove counterions: Take the largest organic fragment
def salt_remover(smiles):
    rmv = rdMolStandardize.LargestFragmentChooser()
    cleaned_smiles = []
    for smi in smiles:
        if "." in smi:
            cleaned_smiles.append(Chem.MolToSmiles(rmv.choose(Chem.MolFromSmiles(smi))))
        else:
            cleaned_smiles.append(smi)
    return cleaned_smiles



In [5]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

def optimize_threshold_from_predictions(labels, probs, thresholds, 
                                    ThOpt_metrics = 'Kappa', N_subsets = 100, 
                                    subsets_size = 0.2, with_replacement = False, random_seed = None):

    """ Optimize the decision threshold based on subsets of the training set.
    The threshold that maximizes the Cohen's kappa coefficient or a ROC-based criterion 
    on the training subsets is chosen as optimal.
    
    Parameters
    ----------
    labels: sequence of ints
        True labels for the training set
    probs: sequence of floats
        predicted probabilities for minority class from the training set 
        (e.g. output from cls.predict_proba(data)[:,1])
    thresholds: list of floats
        List of decision thresholds to screen for classification
    ThOpt_metrics: str
        Optimization metric. Choose between "Kappa" and "ROC"
    N_subsets: int
        Number of training subsets to use in the optimization
    subsets_size: float or int
        Size of the subsets. if float, represents the proportion of the dataset to include in the subsets. 
        If integer, it represents the actual number of instances to include in the subsets. 
    with_replacement: bool
        The subsets are drawn randomly. True to draw the subsets with replacement
    random_seed: int    
        random number to seed the drawing of the subsets
    
    Returns
    ----------
    thresh: float
        Optimal decision threshold for classification
    """
    # seeding
    np.random.seed(random_seed)
    random_seeds = np.random.randint(N_subsets*10, size=N_subsets)  
    
    df_preds = pd.DataFrame({'labels':labels,'probs':probs})
    thresh_names = [str(x) for x in thresholds]
    for thresh in thresholds:
        df_preds[str(thresh)] = [1 if x>=thresh else 0 for x in probs]
    # Optmize the decision threshold based on the Cohen's Kappa coefficient
    if ThOpt_metrics == 'Kappa':
        # pick N_subsets training subsets and determine the threshold that provides the highest kappa on each 
        # of the subsets
        kappa_accum = []
        for i in range(N_subsets):
            if with_replacement:
                if isinstance(subsets_size, float):
                    Nsamples = int(df_preds.shape[0]*subsets_size)
                elif isinstance(subsets_size, int):
                    Nsamples = subsets_size                    
                df_subset = resample(df_preds, replace=True, n_samples = Nsamples, stratify=labels, random_state = random_seeds[i])
                labels_subset = df_subset['labels']
            else:
                df_tmp, df_subset, labels_tmp, labels_subset = train_test_split(df_preds, labels, test_size = subsets_size, stratify = labels, random_state = random_seeds[i])
            kappa_train_subset = []
            for col1 in thresh_names:
                kappa_train_subset.append(metrics.cohen_kappa_score(labels_subset, list(df_subset[col1])))
            kappa_accum.append(kappa_train_subset)
        # determine the threshold that provides the best results on the training subsets
        y_values_median, y_values_std = helper_calc_median_std(kappa_accum)
        opt_thresh = thresholds[np.argmax(y_values_median)]
    # Optmize the decision threshold based on the ROC-curve, as described here https://doi.org/10.1007/s11548-013-0913-8
    elif ThOpt_metrics == 'ROC':
        sensitivity_accum = []
        specificity_accum = []
        # Calculate sensitivity and specificity for a range of thresholds and N_subsets
        for i in range(N_subsets):
            if with_replacement:
                if isinstance(subsets_size, float):
                    Nsamples = int(df_preds.shape[0]*subsets_size)
                elif isinstance(subsets_size, int):
                    Nsamples = subsets_size                    
                df_subset = resample(df_preds, n_samples = Nsamples, stratify=labels, random_state = random_seeds[i])
                labels_subset = list(df_subset['labels'])
            else:
                df_tmp, df_subset, labels_tmp, labels_subset = train_test_split(df_preds, labels, test_size = subsets_size, stratify = labels, random_state = random_seeds[i])
            sensitivity = []
            specificity = []
            for thresh in thresholds:
                scores = [1 if x >= thresh else 0 for x in df_subset['probs']]
                tn, fp, fn, tp = metrics.confusion_matrix(labels_subset, scores, labels=sorted(set(labels))).ravel()
                sensitivity.append(tp/(tp+fn))
                specificity.append(tn/(tn+fp))
            sensitivity_accum.append(sensitivity)
            specificity_accum.append(specificity)
        # determine the threshold that provides the best results on the training subsets
        median_sensitivity, std_sensitivity = helper_calc_median_std(sensitivity_accum)
        median_specificity, std_specificity = helper_calc_median_std(specificity_accum)
        roc_dist_01corner = (2*median_sensitivity*median_specificity)/(median_sensitivity+median_specificity)
        opt_thresh = thresholds[np.argmax(roc_dist_01corner)]
    return opt_thresh

def helper_calc_median_std(specificity):
    # Calculate median and std of the columns of a pandas dataframe
    arr = np.array(specificity)
    y_values_median = np.median(arr,axis=0)
    y_values_std = np.std(arr,axis=0)
    return y_values_median, y_values_std 

## Prepare PubChem assays datasets for DeepChem

Read in PubChem assays:

In [9]:
with gzip.open('data/ChEMBL_PubChem_HTS.csv.gz') as inf:
    pubchem_d = pd.read_csv(inf)
pubchem_d.head()

,canonical_smiles,compound_chembl_id,assay_chembl_id,standard_relation,Mean(standard_value),activity_comment
0,Br.Br.C(c1ccncc1)c2cnc[nH]2,CHEMBL1316355,CHEMBL1614421,=,44668.4,Inconclusive
1,Br.Br.NCCSC(=N)N,CHEMBL1256182,CHEMBL1614249,=,31622.8,Not Active
2,Br.Br.NCCSC(=N)N,CHEMBL1256182,CHEMBL1614364,=,446.7,Not Active
3,Br.Br.NCCSC(=N)N,CHEMBL1256182,CHEMBL1614421,=,17782.8,Inconclusive
4,Br.Br.NCCSC(=N)N,CHEMBL1256182,CHEMBL1794375,NaN,35481.3,active


In [10]:
with open('data/pubchem_data.pkl','rb') as inf:
    pubchem_d,pubchem_assay_lookup = pickle.load(inf)

Prepare dataframe with data from all 8 PubChem datasets:

In [11]:
columns=['canonical_smiles', 'compound_chembl_id', 'assay_chembl_id']
df_pubchem_assays = pd.DataFrame()
for i, assay_id in enumerate(pubchem_assay_lookup):
    assay = pubchem_d.loc[pubchem_d['assay_chembl_id']==assay_id]
    acts = pd.concat((assay.loc[assay['activity_comment'] == 'Active'], 
                      assay.loc[assay['activity_comment'] == 'active']))
    inacts = pd.concat((assay.loc[assay['activity_comment'] == 'inactive'],
                        assay.loc[assay['activity_comment'] == 'inconclusive'], 
                        assay.loc[assay['activity_comment'] == 'Inconclusive'], 
                        assay.loc[assay['activity_comment'] == 'Not Active']))
    acts['canonical_smiles'] = salt_remover(acts['canonical_smiles'])
    inacts['canonical_smiles'] = salt_remover(inacts['canonical_smiles'])
    acts = acts[columns]
    inacts = inacts[columns]
    acts[f'label_{assay_id}'] = 1
    inacts[f'label_{assay_id}'] = 0
    # retrieve compound IDs
    mol_names = list(acts['compound_chembl_id']) + list(inacts['compound_chembl_id'])
    if i==0:
        df_pubchem_assays = pd.concat((acts, inacts))
    else:
        df = pd.concat((acts, inacts))
        df_pubchem_assays = pd.concat((df_pubchem_assays, df), axis=0)




**Labelling for Multitask Classification:**

For the majority of compounds, data are available only for one task. 
For the other tasks where no data is available, compounds are labelled as inactive.

In [12]:
# Deal with duplicate compounds
for i, c1 in enumerate(list(df_pubchem_assays[df_pubchem_assays.compound_chembl_id.duplicated()]['compound_chembl_id'])):
    tmp = df_pubchem_assays.loc[df_pubchem_assays.compound_chembl_id == c1].fillna(method='ffill').fillna(method='bfill')
    if i==0:
        df_no_duplicates = tmp.fillna(0).drop(columns='assay_chembl_id').drop_duplicates(keep='first') 
    else:
        df_no_duplicates = pd.concat((df_no_duplicates, tmp.fillna(0).drop(columns='assay_chembl_id').drop_duplicates(keep='first') ))


df_no_duplicates.reset_index(inplace=True, drop=True)

In [13]:
cmpds_duplicates = list(df_no_duplicates.compound_chembl_id)

In [14]:
df_no_duplicates_part1 = df_pubchem_assays.loc[~df_pubchem_assays.compound_chembl_id.isin(cmpds_duplicates)]
df_no_duplicates_part1.drop(columns='assay_chembl_id', inplace=True)

/Users/carmenesposito/Desktop/Softwares/miniconda3/envs/deepchem-test/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [15]:
df_new = pd.concat((df_no_duplicates, df_no_duplicates_part1.fillna(0)))
df_new.reset_index(inplace=True, drop=True)
df_pubchem_assays_new = df_new.drop_duplicates(keep='first')

In [16]:
df_pubchem_assays_new.head()

,canonical_smiles,compound_chembl_id,label_CHEMBL1794375,label_CHEMBL1614421,label_CHEMBL1614249,label_CHEMBL1614166,label_CHEMBL1614364,label_CHEMBL1613933,label_CHEMBL3214913,label_CHEMBL3215169
0,c1cc2c(cc1-c1csc(NNC3=NCCCCC3)n1)OCCO2,CHEMBL1532195,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,c1ccc(-c2csc(NNC3=NCCCCC3)n2)cc1,CHEMBL1510704,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,c1ccc(-c2csc(Nc3ccc4c(c3)OCCO4)n2)cc1,CHEMBL1410042,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,CC(=O)NNc1nc(-c2ccccc2)cs1,CHEMBL1453036,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CC(=O)Nc1nc(C)c(-c2csc(Nc3ccc(O)cc3)n2)s1,CHEMBL1455993,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## Featurizer

#### Descriptor: Morgan FP with radius 2

In [17]:
featurizer = dc.feat.CircularFingerprint()



In [18]:
pchem_feat = featurizer(list(df_pubchem_assays_new.canonical_smiles))

In [19]:
pchem_labels = np.array(df_pubchem_assays_new[[s for s in df_pubchem_assays_new.columns if 'label' in s]])

In [20]:
dcdf_pubchem = dc.data.NumpyDataset(X=pchem_feat, y=pchem_labels)


### Seeding and Train-Test Split

In [30]:
list_random_seeds_paper = [16, 102, 279, 314, 325, 376, 382, 398, 453, 490 , 
                           10, 133, 181, 202, 269, 304, 317, 392, 429, 447,
                           109, 124, 137, 145, 155, 170, 297, 435, 470, 481,
                           33, 37, 59, 76, 299, 340, 412, 444, 471, 493,
                           48, 82, 132, 175, 191, 253, 264, 364, 399, 478]

In [21]:
random_seed = 22

In [22]:
import tensorflow as tf
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

In [23]:
splitter = dc.splits.RandomStratifiedSplitter()
train_dataset, test_dataset = splitter.train_test_split(dcdf_pubchem, frac_train=0.8, seed = random_seed)

## Train Multitask Classifier

In [24]:
# train model 
n_features = dcdf_pubchem.X.shape[1]
n_tasks = dcdf_pubchem.y.shape[1]
model = dc.models.MultitaskClassifier(n_tasks, n_features)
model.fit(train_dataset, nb_epoch=10)


0.01821889281272888

In [25]:
# predict test set
y_true = test_dataset.y
y_pred = model.predict(test_dataset)

# predict training set
train_y_true = train_dataset.y
train_y_pred = model.predict(train_dataset)


In [ ]:
# evaluate model
auc = []
kappa_th05 = []
confusion_th05 = []
metric = dc.metrics.roc_auc_score
for i in range(n_tasks):
    ### AUC
    try:
        auc_score = metric(dc.metrics.to_one_hot(y_true[:,i]), y_pred[:,i])
        auc.append(auc_score)
    except:
        auc.append(np.nan)
    test_probs = y_pred[:,i][:,1]
    scores = [1 if x>=0.5 else 0 for x in test_probs]
    ### Cohen's kappa
    kappa = dc.metrics.cohen_kappa_score(y_true[:,i], scores)
    kappa_th05.append(kappa)
    ### Confusion Matrix
    tn, fp, fn, tp = metrics.confusion_matrix(y_true[:,i], scores, labels=[0,1]).ravel()
    confusion_th05.append([tn, fp, fn, tp])

# optimize the decision threshold for each dataset and re-evaluate the model
thresholds = np.round(np.arange(0.05,1.00,0.05),2)

kappa_thopt = []
opt_thresholds = []
confusion_thopt = []
for i in range(n_tasks):
    test_probs = y_pred[:,i][:,1]
    train_probs = train_y_pred[:,i][:,1]
    # optimize threshold
    opt_thresh = optimize_threshold_from_predictions(train_y_true[:,i], train_probs, thresholds, random_seed = random_seed)
    opt_thresholds.append(opt_thresh)
    # calculate Cohen's kappa with the optimized threshold
    scores = [1 if x>=opt_thresh else 0 for x in test_probs]
    kappa = dc.metrics.cohen_kappa_score(y_true[:,i], scores)
    kappa_thopt.append(kappa)
    ### Confusion Matrix
    tn, fp, fn, tp = metrics.confusion_matrix(y_true[:,i], scores, labels=[0,1]).ravel()
    confusion_thopt.append([tn, fp, fn, tp])






In [27]:
df_output = pd.DataFrame({'dataset':list(pubchem_assay_lookup.keys()), 'AUC':auc, 'kappa_Th05':kappa_th05, 'kappa_ThOpt':kappa_thopt, 'confusion_Th05':confusion_th05, 'confusion_ThOpt':confusion_thopt, 'ThOpt': opt_thresholds})

In [29]:
df_output

,dataset,AUC,kappa_Th05,kappa_ThOpt,confusion_Th05,confusion_ThOpt,ThOpt
0,CHEMBL1794375,0.705273,0.092442,0.118406,"[32246, 149, 842, 56]","[32019, 376, 806, 92]",0.25
1,CHEMBL1614421,0.893486,0.414908,0.436691,"[31828, 342, 721, 402]","[31725, 445, 663, 460]",0.40
2,CHEMBL1614249,0.712300,0.034083,0.058075,"[33236, 7, 49, 1]","[33227, 16, 48, 2]",0.20
3,CHEMBL1614166,0.681152,0.166500,0.370174,"[33271, 2, 18, 2]","[33271, 2, 15, 5]",0.10
4,CHEMBL1614364,0.757982,0.142237,0.140335,"[33050, 58, 166, 19]","[33022, 86, 164, 21]",0.30
5,CHEMBL1613933,0.999970,-0.000030,0.666653,"[33291, 1, 1, 0]","[33291, 1, 0, 1]",0.05
6,CHEMBL3214913,0.892981,0.274761,0.305905,"[33051, 62, 141, 39]","[33025, 88, 131, 49]",0.30
7,CHEMBL3215169,0.827974,0.213712,0.259146,"[33152, 26, 98, 17]","[33120, 58, 89, 26]",0.15
